### Dependencies

In [1]:
from utils import *
from config import *
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Loading and Preprocessing Training Data

In [2]:
lr_ds = image_dataset_from_directory(
    directory=TRAIN_LR_DIR,
    batch_size=BATCH_SIZE,
    labels=None,
    image_size=IMAGE_SIZE)

hr_ds = image_dataset_from_directory(
    directory=TRAIN_HR_DIR,
    batch_size=BATCH_SIZE,
    labels=None,
    image_size=IMAGE_SIZE)

Found 800 files belonging to 1 classes.
Found 800 files belonging to 1 classes.


In [3]:
normalised_lr = lr_ds.map(NORMALIZATION_FUNC)
normalised_hr = hr_ds.map(NORMALIZATION_FUNC)

train_ds = tf.data.Dataset.zip((normalised_lr,normalised_hr))

### Loading and Preprocessing Validation Data

In [4]:
lr_ds = image_dataset_from_directory(
    directory=VAL_LR_DIR,
    labels=None,
    image_size=IMAGE_SIZE)

hr_ds = image_dataset_from_directory(
    directory=VAL_HR_DIR,
    labels=None,
    image_size=IMAGE_SIZE)

Found 100 files belonging to 1 classes.
Found 100 files belonging to 1 classes.


In [5]:
normalised_lr = lr_ds.map(NORMALIZATION_FUNC)
normalised_hr = hr_ds.map(NORMALIZATION_FUNC)

val_ds = tf.data.Dataset.zip((normalised_lr,normalised_hr))

In [6]:
print(train_ds)
print(val_ds)

<ZipDataset shapes: ((None, 256, 256, 3), (None, 256, 256, 3)), types: (tf.float32, tf.float32)>
<ZipDataset shapes: ((None, 256, 256, 3), (None, 256, 256, 3)), types: (tf.float32, tf.float32)>


### Building the Neural Network

In [7]:
input_layer = layers.Input(shape=IMAGE_SIZE_CHL)

initial_convolutional_layer = layers.Conv2D(64,kernel_size=3,activation="relu",padding="same")(input_layer)

for _ in range(8):
    convolutional_layer = layers.Conv2D(64,kernel_size=3,activation="relu",padding="same")(initial_convolutional_layer)
    x = layers.add([initial_convolutional_layer,convolutional_layer])

output_layer = layers.Conv2D(3,kernel_size=3,activation="sigmoid",padding="same")(x)

model = Model(inputs = input_layer,outputs = output_layer)

### Training the Model

model.compile(loss = LOSS,optimizer=OPTIMIZER,metrics=METRICS)
model.fit(train_ds,
          epochs=EPOCHS,
          validation_data=val_ds,
          callbacks=CALLBACK_TOOL.MODEL_CHECKPOINT(path = MODEL_CHECKPOINT_CALLBACK))

### Testing the model

In [8]:
model = tf.keras.models.load_model(MODEL_CHECKPOINT_CALLBACK)